# Functions Calling

### Setup Functions

In [16]:
from openai import OpenAI
client = OpenAI()

In [62]:
def get_current_temperature(location, unit):
  # print(f"Got following strings - {location}, {unit}")
  return str(57)

def get_rain_probability(location):
  # print(f"Got following strings - {location}")
  return str(0.06)

In [39]:
function_data_1 = {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }


function_data_2 = {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }

In [40]:
tools=[
    {
      "type": "function",
      "function": function_data_1
    },
    {
      "type": "function",
      "function": function_data_2
    }
  ]

### Functions Calling with Chat Completion API

In [93]:
messages = []
messages.append({"role": "system", "content": "You are a weather bot. Use the provided functions to answer questions"})
messages.append({"role": "user", "content": "What's the weather in Toronto today and the likelihood it'll rain?"}) 

response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools
)

In [94]:
response_message = response.choices[0].message.dict()
response_message

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'call_9T3VX0EqixjQtAyRIK9aEZRb',
   'function': {'arguments': '{"location": "Toronto, ON", "unit": "Celsius"}',
    'name': 'get_current_temperature'},
   'type': 'function'},
  {'id': 'call_ChhR81PDFdm8WqfrCumNXwUI',
   'function': {'arguments': '{"location": "Toronto, ON"}',
    'name': 'get_rain_probability'},
   'type': 'function'}],
 'refusal': None}

In [95]:
response_message = response.choices[0].message.dict()
messages.append(response_message)

for idx, function_call in enumerate(response_message["tool_calls"]):
  function_str = function_call["function"]["name"] + "(**"+function_call["function"]["arguments"] + ")"
  print(function_str)
  messages.append({
    "role": "tool",
    "content": eval(function_str),
    "tool_call_id": response_message['tool_calls'][idx]['id']
    })

get_current_temperature(**{"location": "Toronto, ON", "unit": "Celsius"})
get_rain_probability(**{"location": "Toronto, ON"})


In [98]:
response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools
)

In [99]:
response.choices[0].message.dict()


{'content': 'The current temperature in Toronto is 57°C, and there is a 6% likelihood of rain today.',
 'role': 'assistant',
 'function_call': None,
 'tool_calls': None,
 'refusal': None}

### Function Calling With Assistants

In [19]:
assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4o",
  tools=[
    {
      "type": "function",
      "function": function_data_1
    },
    {
      "type": "function",
      "function": function_data_2
    }
  ]
)

In [20]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What's the weather in Toronto today and the likelihood it'll rain?",
)

### Execute a Run

In [21]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [22]:
run.status

'requires_action'

In [23]:
run.dict()

{'id': 'run_0DBHJLwB2ZQT7SgbH15yba1F',
 'assistant_id': 'asst_g98zNgVowf19K7ImXgKrOoto',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1724356135,
 'expires_at': 1724356735,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_uAV7KJFt9aVPbPsmZPbDaoao',
     'function': {'arguments': '{"location": "Toronto, ON", "unit": "Celsius"}',
      'name': 'get_current_temperature'},
     'type': 'function'},
    {'id': 'call_nYT0CK24d4970dY8arXznCUp',
     'function': {'arguments': '{"location": "Toronto, ON"}',
      'name': 'get_rain_probability'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'response_format': 'auto',
 'started_at':

### Extract and Execute Function Calls

In [25]:
for function_call in run.dict()["required_action"]["submit_tool_outputs"]["tool_calls"]:
  print(function_call["function"]["name"] + "(**"+function_call["function"]["arguments"] + ")")
  print(eval(function_call["function"]["name"] + "(**"+function_call["function"]["arguments"] + ")"))

get_current_temperature(**{"location": "Toronto, ON", "unit": "Celsius"})
Got following strings - Toronto, ON, Celsius
57
get_rain_probability(**{"location": "Toronto, ON"})
Got following strings - Toronto, ON
0.06


In [26]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.dict())
else:
  print(run.status)

requires_action


### Feeding Results to Model

In [27]:
# Define the list to store tool outputs
tool_outputs = []

# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_current_temperature":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": eval(tool.function.name  + "(**"+ tool.function.arguments + ")")
    })
  elif tool.function.name == "get_rain_probability":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output":  eval(tool.function.name  + "(**"+ tool.function.arguments + ")")
    })

Got following strings - Toronto, ON, Celsius
Got following strings - Toronto, ON


In [28]:
tool_outputs

[{'tool_call_id': 'call_uAV7KJFt9aVPbPsmZPbDaoao', 'output': '57'},
 {'tool_call_id': 'call_nYT0CK24d4970dY8arXznCUp', 'output': '0.06'}]

In [29]:
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")

Tool outputs submitted successfully.


In [30]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_kL2Uk2WWezz0ffAdESngBe3R', assistant_id='asst_g98zNgVowf19K7ImXgKrOoto', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The current temperature in Toronto is 14°C (57°F), and the likelihood of rain today is 6%.'), type='text')], created_at=1724356471, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_0DBHJLwB2ZQT7SgbH15yba1F', status=None, thread_id='thread_3lrVbl18udfxwEAQIg7nASaG'), Message(id='msg_6AK0wOo3CrMTKBXh8aPWzEce', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="What's the weather in Toronto today and the likelihood it'll rain?"), type='text')], created_at=1724356121, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_3lrVbl18udfxwEAQIg7nASaG')], object='list', first_id

In [38]:
messages.dict()["data"][0]["content"][0]["text"]["value"]

'The current temperature in Toronto is 14°C (57°F), and the likelihood of rain today is 6%.'